In [ ]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
def DEVSTimeToMinutes(aDEVSTime):
    """
    Converts a DEVS timestamp, of the form hours:minutes:seconds:miliseconds:sth
    to a python datetime object, starting with year 1990 on.
    """
    START_YEAR = 1990
    
    splittedDate = aDEVSTime.split(':')
    # Normal assert
    hours = int(splittedDate[0])
    minutes = int(splittedDate[1])
    seconds = int(splittedDate[2])
    mseconds = int(splittedDate[3])
    
    return minutes + hours * 60

import math

"""
    Custom date formatter for x axis in DEVS time format
"""
def format_date(x, pos=None):
    hours = math.floor(x / 60)
    minutes = x - hours*60
    
    return '%02d:%02d:00:00' % (hours,minutes)

# Helper lambdas
daysToHours = lambda d: d*24*60

## Assumptions

It is assumed the the output being parsed was generated with the following top model: **models/controllerAndSolar.ma**. This contains three output ports:
- **grid_out**: Power demanded on sold to the grid. 
- **controlBatteryState**: Battery state changes.
- **controlBatteryDemand**: Power demanded from the controller to the battery.

In [ ]:
outputFile = open('../simulation/src/out/realData', 'r')

parsedOutput = []
for line in outputFile:
    # Trim newline character from read line
    line = line[:-1]
    # Split by spaces
    splittedLine = line.split(' ')
    # Drop excedent emoty read parts
    splittedLine = [readPart for readPart in splittedLine if readPart != '']
    parsedOutput.append(splittedLine)

In [ ]:
parsedData = pd.DataFrame(data=parsedOutput, columns=['time', 'port', 'value'])
parsedData.loc[:, 'value'] = parsedData.value.astype(float)
parsedData.loc[:, 'time'] = parsedData.time.map(DEVSTimeToMinutes)

In [ ]:
parsedData.columns

In [ ]:
parsedData.port.unique()

In [ ]:
gridOutData = parsedData[parsedData.port == 'grid_out']
gridSellData = parsedData[parsedData.port == 'grid_sell']
generatedSolarPower = parsedData[parsedData.port == 'solarpowerbeinggenerated']
generatedWindPower = parsedData[parsedData.port == 'windpowerbeinggenerated']
batteryDemand = parsedData[parsedData.port == 'controlbatterydemand']
batteryState = parsedData[parsedData.port == 'controlbatterystate']

### Plotting update in battery charge

In [ ]:
columnMappers = {
    'time': DEVSTimeToMinutes,
    'charge': float
}
chargeChanges = pd.read_csv('/tmp/batteryChargeChanges', sep=';', usecols=[1,2], names=['time', 'charge'], converters=columnMappers)

In [ ]:
SIMULATION_DAYS = 7
fig, ax = plt.subplots(nrows=2, ncols=1, figsize = (15,10))

ax[0].plot(chargeChanges.time, chargeChanges.charge, color='r', label='Carga')
ax[0].plot(generatedSolarPower.time, generatedSolarPower.value, color='y', label='Potencia generada por panel solar')

ax[0].set_xlim(left=0)

ax[0].set_xlabel('Tiempo [DEVS time]')
ax[0].set_ylabel('Potencia generada / Carga [W / Wh]')

ax[0].legend()

for day in range(SIMULATION_DAYS-1):
    ax[0].axvline(x=24*60*(day+1), color='black', linewidth=1)

ax[1].plot(gridOutData.time, gridOutData.value, label='Potencia pedida a la red')

ax[1].set_xlim(left=0)
ax[1].xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
ax[1].set_xticks(np.arange(daysToHours(1),daysToHours(SIMULATION_DAYS), daysToHours(1)))

ax[1].legend()

fig.autofmt_xdate()

plt.tight_layout()
plt.show()

### Battery state changes according to charge

In [ ]:
SIMULATION_DAYS = 7
fig, ax = plt.subplots(nrows=2, sharex=True, figsize = (15,10))

ax[0].plot(batteryState.time, batteryState.value, '--bo', label='Estado de la batería')

for day in range(SIMULATION_DAYS-1):
    ax[0].axvline(x=24*60*(day+1), color='black', linewidth=1)
    ax[1].axvline(x=24*60*(day+1), color='black', linewidth=1)

ax[0].set_xlim(left=0)
ax[0].xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
ax[0].set_position(matplotlib.transforms.Bbox.from_bounds(0,0,1,.32))
ax[0].set_xticks(np.arange(daysToHours(1),daysToHours(SIMULATION_DAYS), daysToHours(1)))

ax[0].legend()

import matplotlib
# set the position of ax2 so that it is short (y2=0.32) but otherwise the same size as ax
ax[1].set_position(matplotlib.transforms.Bbox.from_bounds(0,-.101,1,.10))
ax[1].fill_between(chargeChanges.time, chargeChanges.charge, color='r', label='Carga')
ax[1].spines['top'].set_visible(False)
ax[1].yaxis.set_visible(False)
ax[1].set_ylim(top=600)


# plt.tight_layout()
plt.ion()
plt.show()

### Analizing enery sold to the grid

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))

ax.bar(gridSellData.time, gridSellData.value)

ax.set_xlim(left=0)
ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
ax.set_xticks(np.arange(daysToHours(1),daysToHours(SIMULATION_DAYS), daysToHours(1)))

plt.show()